# Introduction tutorial

This tutorial is an extended version of the quickstart quide in the README and launches the marginalisation script from the Jupyter notebook interface.

We will assume that you have cloned the repo, created the conda environment from the `environment.yml` file and created your `config_local.ini`.

## Input parameters

### Input and output data paths

To be able to do our example run on the provided data, you need to point the local configfile to your local repository clone:
```ini
[data_paths]
local_path = /Users/<YourUser>/repos/ExoTiC-ISM
```

To run the script on the provided example data on the repository, you can keep the default input data path in the configfile, as it points to the data directory in the respository.
```ini
[data_paths]
...
input_path = ${local_path}/data
```

You will need to specifiy a location on disk for the output data:
```ini
[data_paths]
...
output_path = /Users/MyUser/outputs
```

And if you like, you can change the experiment suffix that will be attached to the filename of your output folder:
```ini
[data_paths]
...
run_name = notebook_test
```

In the end the `[data_paths]` section in the configfile should look something like this:
```ini
[data_paths]
local_path = /Users/MyUser/repos/ExoTiC-ISM
input_path = ${local_path}/data
output_path = /Users/MyUser/outputs
run_name = notebook_test
```

### General setup

For the general setup, we are interested in what instrument and grating was used to collect the data on which specific stellar system. This is done in the configfile section `[setup]`.

First, define what star the data is from - the example data is of Wasp-17b, so we dubbed the stellar system section `W17`:
```ini
[setup]
data_set = W17
```

The instrument for this data is Wide Field Camera 3 (WFC3) and the data was taken with the IR grating G141:
```ini
[setup]
...
instrument = WFC3
grating = G141
```

You need to decide which general grid of systematic models you want to use. For WFC3, this grid consists of 50 distinct systematic models used to fit the data, in all of which the transit depth `rl` and the baseline flux `flux0` are always free parameters. The choice you have to make is which of the variable parameters **epoch**, **inclination**, **MsMpR** and **eccentricity** you want to keep fixed ("frozen") or free ("thawed"):

- `fix_time`: all of them are frozen
- `fit_time`: epoch is thawed, other three are frozen
- `fit_inclin`: inclination is thawed, other three are frozen
- `fit_msmpr`: MsMpR is thawed, other three are frozen
- `fit_ecc`: eccentricity is thawed, other three are frozen
- `fit_all`: eccentricity is frozen, other three are thawed

```ini
[setup]
...
grid_selection = fit_time
```

The repository provides a range of both 1D and 3D limb darkening models, so you need to chose which ones to use:
```ini
[setup]
...
ld_model = 3D
```

And finally, you need to decide whether you want the fit results of each systematic model to be displayed at runtime, which can be helpful for troubleshooting but also annoying when trying to work while the code is running, and whether you want the PDF report with the results to be created. It is highly recommended to always keep this `True`, we included the feature to disable this purely to be able to skip this step if the required python packages for this are not available for any reason.

```ini
[setup]
...
plotting = True
report = True
```

In the end, your general setup should look something like this:
```ini
[setup]
data_set = W17
instrument = WFC3
grating = G141
grid_selection = fit_time
ld_model = 3D
plotting = True
report = True
```

### Smooth model parameters

At convenient points in the code, the marginalisation script will calculate a smooth model of the fit. The following configfile section defines at what resultion it gets calculated and what its range in terms of planet phase it will encapsulate. You might need to adjust especially the `half_range` if your data has a significantly different transit length that the example data. For `W17` we use:

```ini
[smooth_model]
resolution = 0.0001
half_range = 0.2
```

### Stellar and planetary system parameters

The last important bit is to define the input parameters for the star and planet. For each new data set you want to analyse, you need to create a new section in your configfile that carries the same section name like what you put in the `data_set` name in `[section] -> data_set`. For the example data, we have a section called `[W17]` that holds all stellar parameters (for limb darkening) and the initial guesses for the fitting parameters for the planetary transit.

```ini
[W17]
...
rl = 0.12169232
epoch = 57957.970153390
inclin = 87.34635
ecc = 0.0
omega = 0.0
Per = 3.73548535
aor = 7.0780354

; limb darkening parameters
metallicity = -1.0
Teff = 6550
logg = 4.5
```

At the top of this section is where you drop in the file names of your data files, one lightcurve file and one file with a wavelength array. If you have more than one data set available for a specific system, your filenames should indicate by what they differ, e.g. different gratings (e.g. G141 or G102), so that you have to make only one single change in your configfile when your changing from one data set to the other. For example, if you have W17 data both with the G141 as well as the G102 grating, you can include the grating name in the filenames and adjust them in the configfile in such a way that the correct files get picked when you adjust your grating of choice in `[setup] -> grating`.

```ini
[setup]
...
grating = G141
...

[W17]
lightcurve_file = W17_${setup:grating}_lightcurve_test_data.txt
wvln_file = W17_${setup:grating}_wavelength_test_data.txt
```

whic represents these data files:  
```bash
W17_G141_lightcurve_test_data.txt
W17_G141_wavelength_test_data.txt
```